<a href="https://colab.research.google.com/github/evalucre/Economic-Financial-Analysis/blob/main/AlgoTrade_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yahoofinancials

!pip install yahoo_fin
!pip install requests_html

!pip install yfinance 

import numpy as np
import time
import os 
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import pandas as pd
import datetime as datetime
from yahoofinancials import YahooFinancials
import yahoo_fin.stock_info as si
import yfinance as yf

import requests
from math import floor
from termcolor import colored as cl

from google.colab import drive
drive.mount('/content/drive')

plt.rcParams['figure.figsize'] = (20, 10)
plt.style.use('fivethirtyeight')


  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-cp37-none-any.whl size=15192 sha256=437ac1b45110cea24f6c2587224b9d90ae86a728b5f14ecf7caca0cd94586252
  Stored in directory: /root/.cache/pip/wheels/d9/7e/cf/4977a8572d5247242a4b13018d1d36923024ba84236e0d28bc
Successfully built yahoofinancials
     |████████████████████████████████| 81kB 2.0MB/s 
     |████████████████████████████████| 92kB 6.2MB/s 
     |████████████████████████████████| 81kB 6.8MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-cp37-none-any.whl size=6067 sha256=db7ad73ba7c86f6fe9ad7ab754b1407a3359c686bd3c7dbd1bcb2d687c601dc4
  Stored in directory: /root/.cache/pip/wheels/f1/80/5a/444ba08a550cdd241bd9baf8bae44be750efe370adb944506a
  Created wheel for parse: filename=parse-1.19.0-cp37-none-any.whl size=24581 sha256=de91d72c53c4588ad30c29d2726be7ecd2f8b5d70c55b82bb6e26392888cc59f
  Stored in directory: /root/.cache/pip/wheels/c0/39/ea/e2fd67

In [ ]:
def get_ticker_list(grouped_stocks):
  if grouped_stocks == 'dow':
    ticker_list = si.tickers_dow()

  elif  grouped_stocks == 'sp500':
    ticker_list = si.tickers_sp500()

  elif grouped_stocks == 'nasdaq':
    ticker_list = si.tickers_nasdaq()

  elif grouped_stocks == 'tsx':
    ticker_list = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tsx_tickers.csv')
    ticker_list = ticker_list.iloc[:,0].tolist()

  elif grouped_stocks == 'exp':
    ticker_list = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/exp_tickers.csv')  
    ticker_list = ticker_list.iloc[:,0].tolist()  

  elif grouped_stocks == 'tsx60':
    ticker_list = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tsx60_tickers.csv')  
    ticker_list = ticker_list.iloc[:,0].tolist()  

  else:
    ticker_list = "Input group of stocks not recognised"

  return ticker_list




In [ ]:
def get_trading_data(ticker_list, start_date, end_date, index_as_date, interval):
  
  trading_datas = {}
  
  for ticker in ticker_list:
    try:
      trading_datas[ticker] = si.get_data(ticker, start_date, end_date, index_as_date, interval)  
    except:
      pass

  trading_datas = pd.concat(trading_datas)

  trading_datas = trading_datas.reset_index(level=[0,1])\
                               .drop(labels='level_0', axis=1)\
                               .rename(columns={'level_1':'date'})\
                               .set_index('date')

  return trading_datas  


In [ ]:
def get_financial_data(ticker_list):
  
  cashflow_datas = {}
  balncsheet_datas = {}
  incomstment_datas = {}
  
  for ticker in ticker_list:
    cashflow_datas[ticker] = si.get_cash_flow(ticker) 
    balncsheet_datas[ticker] = si.get_balance_sheet(ticker)
    incomstment_datas[ticker] = si.get_income_statement(ticker) 


  cashflow_datas = pd.concat(cashflow_datas) 
  balncsheet_datas = pd.concat(balncsheet_datas)
  incomstment_datas = pd.concat(incomstment_datas)

  cashflow_datas = cashflow_datas.transpose()
  balncsheet_datas = balncsheet_datas.transpose()
  incomstment_datas = incomstment_datas.transpose()

  return cashflow_datas, balncsheet_datas, incomstment_datas 



In [ ]:
def get_selection(trading_datas):
  """
  Select best stocks (numb = 10) with right price volatility and variability for trading 
  or investing. When the preference is to identify stocks for investing, input 
  data should cover a longer time period (years). When trading is the focus, the
  input data should cover shorter time periods (like week, month, or quarters) 

  """

  metric_Pstdv = {}
  metric_Rstdv = {}
  metric_Vmean = {}
  metric_Msize = {}

  trading_datas['prange'] = trading_datas['high'] - trading_datas['low']

  stocks = trading_datas.ticker.unique()
  for stock in stocks:
    dt = trading_datas.loc[(trading_datas.ticker == stock)]
    metric_Pstdv[stock] = dt.close.std()  #price standard deviation (high for investment stocks) - profitability indicator
    metric_Rstdv[stock] = dt.prange.std()  #prange standard deviation (high for trading stocks) - volatility indicator
    metric_Vmean[stock] = dt.volume.mean()  #mean of volume - liquidity indicator
    metric_Msize[stock] = (dt.volume.mean())*(dt.close.mean())  #measure of market size/capitalization

#Rank stocks using calculated metrics
  metric_Pstdv = pd.DataFrame(metric_Pstdv, index=[0])
  metric_Rstdv = pd.DataFrame(metric_Rstdv, index=[0])
  metric_Vmean = pd.DataFrame(metric_Vmean, index=[0])
  metric_Msize = pd.DataFrame(metric_Msize, index=[0])

  metric_Pstdv = metric_Pstdv.T.reset_index()\
                               .rename(columns={'index':'ticker'})\
                               .rename(columns={0:'Pstdv'})\
                               .sort_values(by=['Pstdv'], ascending = False)
                            
  
  metric_Rstdv = metric_Rstdv.T.reset_index()\
                               .rename(columns={'index':'ticker'})\
                               .rename(columns={0:'Rstdv'})\
                               .sort_values(by=['Rstdv'], ascending = False)
 

  metric_Vmean = metric_Vmean.T.reset_index()\
                               .rename(columns={'index':'ticker'})\
                               .rename(columns={0:'Vmean'})\
                               .sort_values(by=['Vmean'], ascending = False)


  metric_Msize = metric_Msize.T.reset_index()\
                               .rename(columns={'index':'ticker'})\
                               .rename(columns={0:'Msize'})\
                               .sort_values(by=['Msize'], ascending = False)
                          
#Select top 10 stocks with highest values of the metrics
  numb = 50
  metric_Pstdv = metric_Pstdv[:numb]['ticker'].reset_index(drop=True)      
  metric_Rstdv = metric_Rstdv[:numb]['ticker'].reset_index(drop=True)
  metric_Vmean = metric_Vmean[:numb]['ticker'].reset_index(drop=True)
  metric_Msize = metric_Msize[:numb]['ticker'].reset_index(drop=True) 

  volatl_select = metric_Rstdv

  tradin_select = metric_Rstdv[(pd.Series(metric_Rstdv.to_numpy()).isin(metric_Vmean.to_numpy()))]
  invest_select = metric_Pstdv[(pd.Series(metric_Pstdv.to_numpy()).isin(metric_Vmean.to_numpy()))]
  
  #Definitions:
  #tradin_select - combines volatility and high trade volumes
  #invest_select - long term growth and high capitalization
  #volatl_select - high volatility but does not necessarily have trade volumes

  return tradin_select, invest_select, volatl_select


In [ ]:
def get_macd(trading_datas, select_ticker, fastm, slowm, smooth):

  df = trading_datas[trading_datas['ticker'] == select_ticker]

  exp1 = df.close.ewm(span=fastm, adjust=False).mean()
  exp2 = df.close.ewm(span=slowm, adjust=False).mean()
  macd = pd.DataFrame(exp1 - exp2).rename(columns = {'close':'macd'})
  signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
  hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns = {0:'hist'})
  
  df = pd.concat([df, macd, signal, hist], join = 'inner', axis = 1)

  return df

In [ ]:
def get_smap(trading_datas, select_ticker, roll_wks1, roll_wks2, roll_wks10, roll_wks30, roll_wks40):

  df = trading_datas[trading_datas['ticker'] == select_ticker]

  smap_wks1 = df.close.rolling(window=roll_wks1).mean()
  smap_wks1 = pd.DataFrame(smap_wks1).rename(columns = {'close':'smap_wks1'})

  smap_wks2 = df.close.rolling(window=roll_wks2).mean()
  smap_wks2 = pd.DataFrame(smap_wks2).rename(columns = {'close':'smap_wks2'})

  smap_wks10 = df.close.rolling(window=roll_wks10).mean()
  smap_wks10 = pd.DataFrame(smap_wks10).rename(columns = {'close':'smap_wks10'})

  smap_wks30 = df.close.rolling(window=roll_wks30).mean()
  smap_wks30 = pd.DataFrame(smap_wks30).rename(columns = {'close':'smap_wks30'})

  smap_wks40 = df.close.rolling(window=roll_wks40).mean()
  smap_wks40 = pd.DataFrame(smap_wks40).rename(columns = {'close':'smap_wks40'})

  df = pd.concat([df, smap_wks1, smap_wks2, smap_wks10, smap_wks30, smap_wks40], join = 'inner', axis = 1)
  
  return df

In [ ]:
def get_smav(trading_datas, select_ticker, roll_wks1, roll_wks2, roll_wks10, roll_wks30, roll_wks40):

  df = trading_datas[trading_datas['ticker'] == select_ticker]

  smav_wks1 = df.volume.rolling(window=roll_wks1).mean()
  smav_wks1 = pd.DataFrame(smav_wks1).rename(columns = {'volume':'smav_wks1'})

  smav_wks2 = df.volume.rolling(window=roll_wks2).mean()
  smav_wks2 = pd.DataFrame(smav_wks2).rename(columns = {'volume':'smav_wks2'})

  smav_wks10 = df.volume.rolling(window=roll_wks10).mean()
  smav_wks10 = pd.DataFrame(smav_wks10).rename(columns = {'volume':'smav_wks10'})

  smav_wks30 = df.volume.rolling(window=roll_wks30).mean()
  smav_wks30 = pd.DataFrame(smav_wks30).rename(columns = {'volume':'smav_wks30'})

  smav_wks40 = df.volume.rolling(window=roll_wks40).mean()
  smav_wks40 = pd.DataFrame(smav_wks40).rename(columns = {'volume':'smav_wks40'})

  df = pd.concat([df, smav_wks1, smav_wks2, smav_wks10, smav_wks30, smav_wks40], join = 'inner', axis = 1)

  return df

In [ ]:
 def get_stochst(trading_datas, select_ticker, roll_low, roll_high, fasts, slows):
    """
    Fast stochastic calculation
    %K = (Current Close - Lowest Low)/
    (Highest High - Lowest Low) * 100
    %D = SMA of %K

    Slow stochastic calculation
    %K = %D of fast stochastic
    %D = SMA of %K

    When %K crosses above %D, buy signal 
    When the %K crosses below %D, sell signal
    """
# It seems like d_fast would be best for high volatility trading and d_slow for lower volatility investing

    df = trading_datas[trading_datas['ticker'] == select_ticker]

# Set minimum low and maximum high of the k stoch
    low_min  = df.low.rolling( window = roll_low ).min()
    high_max = df.high.rolling( window = roll_high ).max()

# Fast Stochastic
    df['k_fast'] = 100 * (df.close - low_min)/(high_max - low_min)
    df['d_fast'] = df['k_fast'].rolling(window = fasts).mean()

# Slow Stochastic
    df['k_slow'] = df['d_fast']
    df['d_slow'] = df['k_slow'].rolling(window = slows).mean()

    return df


In [ ]:
def get_boband(trading_datas, select_ticker, roll_period):
#Bollinger Band Algorithm: it seems this is designed for live market decision-making
#The rolling period can be the last 20 trading periods

  df = trading_datas[trading_datas['ticker'] == select_ticker]

  df['Mband'] = df['close'].rolling(window = roll_period).mean()                       #middle band
  df['Uband'] = df['Mband'] + 1.96*df['close'].rolling(window = roll_period).std()     #upper band
  df['Lband'] = df['Mband'] - 1.96*df['close'].rolling(window = roll_period).std()     #lower band

  return df

In [ ]:
def strategy_macd(macd_data):    
    buy_price = []
    sell_price = []
    macd_signal = []
    position = np.zeros(len(macd_data))
    signal = 0

    for i in range(len(macd_data)):
        if macd_data['macd'][i] > macd_data['signal'][i]:
            if signal != 1:
                buy_price.append(macd_data['close'][i])
                sell_price.append(np.nan)
                signal = 1
                macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                macd_signal.append(0)
        elif macd_data['macd'][i] < macd_data['signal'][i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(macd_data['close'][i])
                signal = -1
                macd_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                macd_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            macd_signal.append(0)

    for i in range(len(macd_data)):
      if macd_signal[i] == 1:
        position[i] = 1
      elif macd_signal[i] == -1:
        position[i] = 0
      elif (macd_signal[i] == 0) and (i == 0):
        position[i] = 0
      else:
        position[i] = position[i-1]

    macd_signal = pd.DataFrame(macd_signal).rename(columns = {0:'macd_signal'}).set_index(macd_data.index)
    position = pd.DataFrame(position).rename(columns = {0:'macd_position'}).set_index(macd_data.index)
    buy_price = pd.DataFrame(buy_price).rename(columns = {0:'buy_price'}).set_index(macd_data.index)
    sell_price = pd.DataFrame(sell_price).rename(columns = {0:'sell_price'}).set_index(macd_data.index)

    macd_decisions = pd.concat([macd_data['close'], macd_data['macd'], macd_data['signal'], macd_signal, position, buy_price, sell_price], join = 'inner', axis = 1)


    return macd_decisions

In [ ]:
def strategy_smap(smap_data):    
    buy_price = []
    sell_price = []
    smap_signal = []
    position = np.zeros(len(smap_data))
    signal1, signal2, signal3, signal4, signal5, signal6, signal7, signal8 = [], [], [], [], [], [], [], []
    
#Stock price is above both the 150-day (30-week) and the 200-day (40-week) moving average price lines.   
    signal = 0
    for i in range(len(smap_data)):
        if (smap_data['close'][i] > smap_data['smap_wks30'][i]) and (smap_data['close'][i] > smap_data['smap_wks40'][i]):
            if signal != 1:
                signal = 1
                signal1.append(signal)
            else:
                signal1.append(0)
        elif (smap_data['close'][i] < smap_data['smap_wks30'][i]) and (smap_data['close'][i] < smap_data['smap_wks40'][i]):
            if signal != -1:
                signal = -1
                signal1.append(signal)
            else:
                signal1.append(0)
        else:
            signal1.append(0)

#The 150-day moving average is above the 200-day moving average.
    signal = 0
    for i in range(len(smap_data)):
        if smap_data['smap_wks30'][i] > smap_data['smap_wks40'][i]:
            if signal != 1:
                signal = 1
                signal2.append(signal)
            else:
                signal2.append(0)
        elif smap_data['smap_wks30'][i] < smap_data['smap_wks40'][i]:
            if signal != -1:
                signal = -1
                signal2.append(signal)
            else:
                signal2.append(0)
        else:
            signal2.append(0)

#The 200-day moving average line is trending up for at least 1-month (preferably 4 to 5 months or longer).
    signal = 0
    smooth = 20    #smooth specifies the period over which trend is calculated - we are using 1 month (4 weeks) as default
    trend_wks40 = smap_data.smap_wks40.rolling(window=smooth).mean()
    trend_wks40 = pd.DataFrame(trend_wks40).rename(columns = {'smap_wks40':'trend_wks40'})
    smap_data = pd.concat([smap_data, trend_wks40], join = 'inner', axis = 1)
    signal3.append(0)   #signal3[0] = 0
    for i in range(1, len(smap_data)):
        if (smap_data['trend_wks40'][i] - smap_data['trend_wks40'][i-1]) > 0:
            if signal != 1:
                signal = 1
                signal3.append(signal)
            else:
                signal3.append(0)
        elif (smap_data['trend_wks40'][i] - smap_data['trend_wks40'][i-1]) < 0:
            if signal != -1:
                signal = -1
                signal3.append(signal)
            else:
                signal3.append(0)
        else:
            signal3.append(0)

#The 50-day (10-week moving average) is above both the 150-day and the 200-day moving averages.
    signal = 0
    for i in range(len(smap_data)):
        if (smap_data['smap_wks10'][i] > smap_data['smap_wks30'][i]) and (smap_data['smap_wks10'][i] > smap_data['smap_wks40'][i]):
            if signal != 1:
                signal = 1
                signal4.append(signal)
            else:
                signal4.append(0)
        elif (smap_data['smap_wks10'][i] < smap_data['smap_wks30'][i]) and (smap_data['smap_wks10'][i] < smap_data['smap_wks40'][i]):
            if signal != -1:
                signal = -1
                signal4.append(signal)
            else:
                signal4.append(0)
        else:
            signal4.append(0)

#The current stock price is at least 25 percent above its 52-week low. (Many of the best selections will be 100 percent, 300 percent, or more above their 52-week consolidation period and mount a large-scale advance).
    signal = 0
    min_wks52 = smap_data['close'].min()   #I decided not to specify 52 week minimum since this would be run for various periods, if not this would be a NA when the data is less than 52 weeks
    min_band = 1.25*min_wks52
    for i in range(len(smap_data)):
        if smap_data['close'][i] >= min_band:
            if signal != 1:
                signal = 1
                signal5.append(signal)
            else:
                signal5.append(0)
        elif smap_data['close'][i] < min_band:
            if signal != -1:
                signal = -1
                signal5.append(signal)
            else:
                signal5.append(0)
        else:
            signal5.append(0)

#The current stock price is within at least 25 percent of its 52-week high (the closer to a new high the better).
    signal = 0
    max_wks52 = smap_data['close'].max()   #I decided not to specify 52 week maximum since this would be run for various periods, if not this would be a NA when the data is less than 52 weeks
    max_band = 0.75*max_wks52
    for i in range(len(smap_data)):
        if smap_data['close'][i] >= max_band:
            if signal != 1:
                signal = 1
                signal6.append(signal)
            else:
                signal6.append(0)
        elif smap_data['close'][i] < max_band:
            if signal != -1:
                signal = -1
                signal6.append(signal)
            else:
                signal6.append(0)
        else:
            signal6.append(0)

#The current price is trading above the 50-day moving average as the stock is coming out of a base
    signal = 0
    for i in range(len(smap_data)):
        if smap_data['close'][i] > smap_data['smap_wks10'][i]:
            if signal != 1:
                signal = 1
                signal7.append(signal)
            else:
                signal7.append(0)
        elif smap_data['close'][i] < smap_data['smap_wks10'][i]:
            if signal != -1:
                signal = -1
                signal7.append(signal)
            else:
                signal7.append(0)
        else:
            signal7.append(0)

#The 1 week moving average is trading above the 2 weeks moving average indicating bullish short term trading signal
    signal = 0
    for i in range(len(smap_data)):
        if smap_data['smap_wks1'][i] > smap_data['smap_wks2'][i]:
            if signal != 1:
                signal = 1
                signal8.append(signal)
            else:
                signal8.append(0)
        elif smap_data['smap_wks1'][i] < smap_data['smap_wks2'][i]:
            if signal != -1:
                signal = -1
                signal8.append(signal)
            else:
                signal8.append(0)
        else:
            signal8.append(0)

#Combine all eight signals to get the unified smap signal satisfying all conditions
    signal = 0
    for i in range(len(smap_data)):
        if ((signal1[i]==1 and signal2[i]==1) and (signal3[i]==1 and signal4[i]==1)) and ((signal5[i]==1 and signal6[i]==1) and (signal7[i]==1 and signal8[i]==1)):
            if signal != 1:
                buy_price.append(smap_data['close'][i])
                sell_price.append(np.nan)
                signal = 1
                smap_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                smap_signal.append(0)
        elif ((signal1[i]==-1 and signal2[i]==-1) and (signal3[i]==-1 and signal4[i]==-1)) and ((signal5[i]==-1 and signal6[i]==-1) and (signal7[i]==-1 and signal8[i]==-1)):
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(smap_data['close'][i])
                signal = -1
                smap_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                smap_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            smap_signal.append(0)

#Evaluate stock position based on the combined signals
    for i in range(len(smap_data)):
      if smap_signal[i] == 1:
        position[i] = 1
      elif smap_signal[i] == -1:
        position[i] = 0
      elif (smap_signal[i] == 0) and (i == 0):
        position[i] = 0
      else:
        position[i] = position[i-1]

    smap_signal = pd.DataFrame(smap_signal).rename(columns = {0:'smap_signal'}).set_index(smap_data.index)
    position = pd.DataFrame(position).rename(columns = {0:'smap_position'}).set_index(smap_data.index)
    buy_price = pd.DataFrame(buy_price).rename(columns = {0:'buy_price'}).set_index(smap_data.index)
    sell_price = pd.DataFrame(sell_price).rename(columns = {0:'sell_price'}).set_index(smap_data.index)

    signal1 = pd.DataFrame(signal1).rename(columns = {0:'signal1'}).set_index(smap_data.index)
    signal2 = pd.DataFrame(signal2).rename(columns = {0:'signal2'}).set_index(smap_data.index)
    signal3 = pd.DataFrame(signal3).rename(columns = {0:'signal3'}).set_index(smap_data.index)
    signal4 = pd.DataFrame(signal4).rename(columns = {0:'signal4'}).set_index(smap_data.index)
    signal5 = pd.DataFrame(signal5).rename(columns = {0:'signal5'}).set_index(smap_data.index)
    signal6 = pd.DataFrame(signal6).rename(columns = {0:'signal6'}).set_index(smap_data.index)
    signal7 = pd.DataFrame(signal7).rename(columns = {0:'signal7'}).set_index(smap_data.index)
    signal8 = pd.DataFrame(signal8).rename(columns = {0:'signal8'}).set_index(smap_data.index)

    smap_decisions = pd.concat([smap_data['close'], smap_signal, position, buy_price, sell_price, signal1, signal2, signal3, signal4, signal5, signal6, signal7, signal8], join = 'inner', axis = 1)


    return smap_decisions

In [ ]:
def strategy_smav(smav_data):    
    buy_price = []
    sell_price = []
    smav_signal = []
    position = np.zeros(len(smav_data))

    tradv_signal = []
    
#If volume is above the 50-day moving average, that is a bullish signal. 
    signal = 0
    for i in range(len(smav_data)):
        if smav_data['volume'][i] > smav_data['smav_wks10'][i]:
            if signal != 1:
                buy_price.append(smav_data['close'][i])
                sell_price.append(np.nan)
                signal = 1
                smav_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                smav_signal.append(0)
        elif smav_data['volume'][i] < smav_data['smav_wks10'][i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(smav_data['close'][i])
                signal = -1
                smav_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                smav_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            smav_signal.append(0)

#For day-trading purposes, if volume is above the 10-day moving average, that is a bullish signal - NOTE: I added this signal for trading purposes on high volatility stocks
    signal = 0
    for i in range(len(smav_data)):
        if smav_data['volume'][i] > smav_data['smav_wks2'][i]:
            if signal != 1:
                signal = 1
                tradv_signal.append(signal)
            else:
                tradv_signal.append(0)
        elif smav_data['volume'][i] < smav_data['smav_wks2'][i]:
            if signal != -1:
                signal = -1
                tradv_signal.append(signal)
            else:
                tradv_signal.append(0)
        else:
            tradv_signal.append(0)

#Evaluate stock position based on the smav signals
    for i in range(len(smav_data)):
      if smav_signal[i] == 1:
        position[i] = 1
      elif smav_signal[i] == -1:
        position[i] = 0
      elif (smav_signal[i] == 0) and (i == 0):
        position[i] = 0
      else:
        position[i] = position[i-1]

    smav_signal = pd.DataFrame(smav_signal).rename(columns = {0:'smav_signal'}).set_index(smav_data.index)
    position = pd.DataFrame(position).rename(columns = {0:'smav_position'}).set_index(smav_data.index)
    buy_price = pd.DataFrame(buy_price).rename(columns = {0:'buy_price'}).set_index(smav_data.index)
    sell_price = pd.DataFrame(sell_price).rename(columns = {0:'sell_price'}).set_index(smav_data.index)

    tradv_signal = pd.DataFrame(tradv_signal).rename(columns = {0:'tradv_signal'}).set_index(smav_data.index)

    smav_decisions = pd.concat([smav_data['close'], smav_signal, position, buy_price, sell_price, tradv_signal], join = 'inner', axis = 1)


    return smav_decisions

In [ ]:
def strategy_stochst(stochst_data, oversold=20, overbought=80):    
    buy_price = []
    sell_price = []
    stochst_signal = []
    position = np.zeros(len(stochst_data))
    signal = 0

    for i in range(len(stochst_data)):
        if stochst_data['d_slow'][i] < oversold:
            if signal != 1:
                buy_price.append(stochst_data['close'][i])
                sell_price.append(np.nan)
                signal = 1
                stochst_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                stochst_signal.append(0)
        elif stochst_data['d_slow'][i] > overbought:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(stochst_data['close'][i])
                signal = -1
                stochst_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                stochst_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            stochst_signal.append(0)

    for i in range(len(stochst_data['close'])):
      if stochst_signal[i] == 1:
        position[i] = 1
      elif stochst_signal[i] == -1:
        position[i] = 0
      elif (stochst_signal[i] == 0) and (i == 0):
        position[i] = 0
      else:
        position[i] = position[i-1]

    stochst_signal = pd.DataFrame(stochst_signal).rename(columns = {0:'stochst_signal'}).set_index(stochst_data.index)
    position = pd.DataFrame(position).rename(columns = {0:'stochst_position'}).set_index(stochst_data.index)
    buy_price = pd.DataFrame(buy_price).rename(columns = {0:'buy_price'}).set_index(stochst_data.index)
    sell_price = pd.DataFrame(sell_price).rename(columns = {0:'sell_price'}).set_index(stochst_data.index)

    stochst_decisions = pd.concat([stochst_data['close'], stochst_data['d_fast'], stochst_data['d_slow'], stochst_signal, position, buy_price, sell_price], join = 'inner', axis = 1)


    return stochst_decisions

In [ ]:
def strategy_boband(boband_data):    
    buy_price = []
    sell_price = []
    boband_signal = []
    position = np.zeros(len(boband_data))
    signal = 0

    for i in range(len(boband_data)):
        if boband_data['close'][i] == boband_data['Lband'][i]:
            if signal != 1:
                buy_price.append(boband_data['close'][i])
                sell_price.append(np.nan)
                signal = 1
                boband_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                boband_signal.append(0)
        elif boband_data['close'][i] == boband_data['Uband'][i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(boband_data['close'][i])
                signal = -1
                boband_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                boband_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            boband_signal.append(0)

    for i in range(len(boband_data)):
      if boband_signal[i] == 1:
        position[i] = 1
      elif boband_signal[i] == -1:
        position[i] = 0
      elif (boband_signal[i] == 0) and (i == 0):
        position[i] = 0
      else:
        position[i] = position[i-1]

    boband_signal = pd.DataFrame(boband_signal).rename(columns = {0:'boband_signal'}).set_index(boband_data.index)
    position = pd.DataFrame(position).rename(columns = {0:'boband_position'}).set_index(boband_data.index)
    buy_price = pd.DataFrame(buy_price).rename(columns = {0:'buy_price'}).set_index(boband_data.index)
    sell_price = pd.DataFrame(sell_price).rename(columns = {0:'sell_price'}).set_index(boband_data.index)

    boband_decisions = pd.concat([boband_data['close'], boband_data['Lband'], boband_data['Uband'], boband_signal, position, buy_price, sell_price], join = 'inner', axis = 1)


    return boband_decisions

In [ ]:
def scenarios_invest(macd_decisions, smap_decisions, smav_decisions, stochst_decisions, boband_decisions):
  """
  Investment Scenarios
  -High Risk (HR): based on only smap and smav 
  -Medium Risk (MR): based on smap, smav, macd, and stochastic
  -Low Risk (LR): based on smap, smav, macd, stochastic, and bollinger
  """

  df = pd.concat([macd_decisions, smap_decisions, smav_decisions, stochst_decisions, boband_decisions], join = 'inner', axis = 1)

  HR_signal = []
  MR_signal = []
  LR_signal = []
  HR_position = np.zeros(len(df))
  MR_position = np.zeros(len(df))
  LR_position = np.zeros(len(df))


#HR SIGNAL CONDITIONS
  signal = 0
  for i in range(len(df)):
      if (df['smap_signal'][i] == 1) and (df['smav_signal'][i] == 1):
          if signal != 1:
              signal = 1
              HR_signal.append(signal)
          else:
              HR_signal.append(0)
      elif (df['smap_signal'][i] == -1) and (df['smav_signal'][i] == -1):
          if signal != -1:
              signal = -1
              HR_signal.append(signal)
          else:
              HR_signal.append(0)
      else:
          HR_signal.append(0)

#MR SIGNAL CONDITIONS
  signal = 0
  for i in range(len(df)):
      if ((df['smap_signal'][i] == 1) and (df['smav_signal'][i] == 1)) and ((df['macd_signal'][i] == 1) and (df['stochst_signal'][i] == 1)):
          if signal != 1:
              signal = 1
              MR_signal.append(signal)
          else:
              MR_signal.append(0)
      elif ((df['smap_signal'][i] == -1) and (df['smav_signal'][i] == -1)) and ((df['macd_signal'][i] == -1) and (df['stochst_signal'][i] == -1)):
          if signal != -1:
              signal = -1
              MR_signal.append(signal)
          else:
              MR_signal.append(0)
      else:
          MR_signal.append(0)

#LR SIGNAL CONDITIONS
  signal = 0
  for i in range(len(df)):
      if (((df['smap_signal'][i] == 1) and (df['smav_signal'][i] == 1)) and ((df['macd_signal'][i] == 1) and (df['stochst_signal'][i] == 1))) and (df['boband_signal'][i] == 1):
          if signal != 1:
              signal = 1
              LR_signal.append(signal)
          else:
              LR_signal.append(0)
      elif (((df['smap_signal'][i] == -1) and (df['smav_signal'][i] == -1)) and ((df['macd_signal'][i] == -1) and (df['stochst_signal'][i] == -1))) and (df['boband_signal'][i] == -1):
          if signal != -1:
              signal = -1
              LR_signal.append(signal)
          else:
              LR_signal.append(0)
      else:
          LR_signal.append(0)


#HR POSITION CONDITIONS
  for i in range(len(df)):
    if HR_signal[i] == 1:
      HR_position[i] = 1
    elif HR_signal[i] == -1:
      HR_position[i] = 0
    elif (HR_signal[i] == 0) and (i == 0):
      HR_position[i] = 0
    else:
      HR_position[i] = HR_position[i-1]

#MR POSITION CONDITIONS
  for i in range(len(df)):
    if MR_signal[i] == 1:
      MR_position[i] = 1
    elif MR_signal[i] == -1:
      MR_position[i] = 0
    elif (MR_signal[i] == 0) and (i == 0):
      MR_position[i] = 0
    else:
      MR_position[i] = MR_position[i-1]

#LR POSITION CONDITIONS
  for i in range(len(df)):
    if LR_signal[i] == 1:
      LR_position[i] = 1
    elif LR_signal[i] == -1:
      LR_position[i] = 0
    elif (LR_signal[i] == 0) and (i == 0):
      LR_position[i] = 0
    else:
      LR_position[i] = LR_position[i-1]


  HR_signal = pd.DataFrame(HR_signal).rename(columns = {0:'HR_signal'}).set_index(df.index)
  MR_signal = pd.DataFrame(MR_signal).rename(columns = {0:'MR_signal'}).set_index(df.index)
  LR_signal = pd.DataFrame(LR_signal).rename(columns = {0:'LR_signal'}).set_index(df.index)

  HR_position = pd.DataFrame(HR_position).rename(columns = {0:'HR_position'}).set_index(df.index)
  MR_position = pd.DataFrame(MR_position).rename(columns = {0:'MR_position'}).set_index(df.index)
  LR_position = pd.DataFrame(LR_position).rename(columns = {0:'LR_position'}).set_index(df.index)

# invest_table = pd.concat([df['close'], HR_signal, MR_signal, LR_signal, HR_position, MR_position, LR_position], join = 'inner', axis = 1)
  invest_table = pd.concat([macd_decisions['close'], HR_signal, MR_signal, LR_signal, HR_position, MR_position, LR_position], join = 'inner', axis = 1)

  return invest_table

In [ ]:
def scenarios_tradin(macd_decisions, smap_decisions, smav_decisions, stochst_decisions, boband_decisions):

  """
  Trading Scenarios (all considered high risk)
  -Short View (SV): uses one and two weeks trends (5 and 10 days) ...smap, smav, boband
  -Medium View (MV): uses one and two weeks trends (5 and 10 days)...smap, smav, macd, stochst
  -Long View (LV): uses current price and ten weeks trends (close and 50 days)...smap, smav, macd, boband
  """


  df = pd.concat([macd_decisions, smap_decisions, smav_decisions, stochst_decisions, boband_decisions], join = 'inner', axis = 1)

  SV_signal = []
  MV_signal = []
  LV_signal = []
  SV_position = np.zeros(len(df))
  MV_position = np.zeros(len(df))
  LV_position = np.zeros(len(df))

  #SV SIGNAL CONDITIONS
  signal = 0
  for i in range(len(df)):
      if ((df['signal8'][i] == 1) and (df['tradv_signal'][i] == 1)) and (df['boband_signal'][i] == 1):
          if signal != 1:
              signal = 1
              SV_signal.append(signal)
          else:
              SV_signal.append(0)
      elif ((df['signal8'][i] == -1) and (df['tradv_signal'][i] == -1)) and (df['boband_signal'][i] == -1):
          if signal != -1:
              signal = -1
              SV_signal.append(signal)
          else:
              SV_signal.append(0)
      else:
          SV_signal.append(0)

  #MV SIGNAL CONDITIONS
  signal = 0
  for i in range(len(df)):
      if ((df['signal8'][i] == 1) and (df['tradv_signal'][i] == 1)) and ((df['macd_signal'][i] == 1) and (df['stochst_signal'][i] == 1)):
          if signal != 1:
              signal = 1
              MV_signal.append(signal)
          else:
              MV_signal.append(0)
      elif ((df['signal8'][i] == -1) and (df['tradv_signal'][i] == -1)) and ((df['macd_signal'][i] == -1) and (df['stochst_signal'][i] == -1)):
          if signal != -1:
              signal = -1
              MV_signal.append(signal)
          else:
              MV_signal.append(0)
      else:
          MV_signal.append(0)

  #LV SIGNAL CONDITIONS
  signal = 0
  for i in range(len(df)):
      if ((df['signal7'][i] == 1) and (df['smav_signal'][i] == 1)) and ((df['macd_signal'][i] == 1) and (df['boband_signal'][i] == 1)):
          if signal != 1:
              signal = 1
              LV_signal.append(signal)
          else:
              LV_signal.append(0)
      elif ((df['signal7'][i] == -1) and (df['smav_signal'][i] == -1)) and ((df['macd_signal'][i] == -1) and (df['boband_signal'][i] == -1)):
          if signal != -1:
              signal = -1
              LV_signal.append(signal)
          else:
              LV_signal.append(0)
      else:
          LV_signal.append(0)


#SV POSITION CONDITIONS
  for i in range(len(df)):
    if SV_signal[i] == 1:
      SV_position[i] = 1
    elif SV_signal[i] == -1:
      SV_position[i] = 0
    elif (SV_signal[i] == 0) and (i == 0):
      SV_position[i] = 0
    else:
      SV_position[i] = SV_position[i-1]

#MV POSITION CONDITIONS
  for i in range(len(df)):
    if MV_signal[i] == 1:
      MV_position[i] = 1
    elif MV_signal[i] == -1:
      MV_position[i] = 0
    elif (MV_signal[i] == 0) and (i == 0):
      MV_position[i] = 0
    else:
      MV_position[i] = MV_position[i-1]

#LV POSITION CONDITIONS
  for i in range(len(df)):
    if LV_signal[i] == 1:
      LV_position[i] = 1
    elif LV_signal[i] == -1:
      LV_position[i] = 0
    elif (LV_signal[i] == 0) and (i == 0):
      LV_position[i] = 0
    else:
      LV_position[i] = LV_position[i-1]




  SV_signal = pd.DataFrame(SV_signal).rename(columns = {0:'SV_signal'}).set_index(df.index)
  MV_signal = pd.DataFrame(MV_signal).rename(columns = {0:'MV_signal'}).set_index(df.index)
  LV_signal = pd.DataFrame(LV_signal).rename(columns = {0:'LV_signal'}).set_index(df.index)

  SV_position = pd.DataFrame(SV_position).rename(columns = {0:'SV_position'}).set_index(df.index)
  MV_position = pd.DataFrame(MV_position).rename(columns = {0:'MV_position'}).set_index(df.index)
  LV_position = pd.DataFrame(LV_position).rename(columns = {0:'LV_position'}).set_index(df.index)

  #tradin_table = pd.concat([df['close'], SV_signal, MV_signal, LV_signal, SV_position, MV_position, LV_position], join = 'inner', axis = 1)  
  tradin_table = pd.concat([macd_decisions['close'], SV_signal, MV_signal, LV_signal, SV_position, MV_position, LV_position], join = 'inner', axis = 1)

  return tradin_table


In [ ]:
"""
WORKFLOW EXECUTION SCRIPT: THIS IS USED TO RUN THE TRADING MODULES ON OFFLINE BASIS

"""
#Selection of stocks for trading or investment
grouped_stocks='tsx'
ticker_list = get_ticker_list(grouped_stocks)                                             #Other currently available groups are: 'tsx', 'sp500', 'nasdaq' 

#Get (historical) data on longterm performance os stock - 20 years preferable
start_date = (datetime.date.today() + datetime.timedelta(days=-3)).strftime("%m/%d/%Y")                                                            #Input format is: month/day/Year
end_date = datetime.date.today().strftime("%m/%d/%Y")
trading_datas = get_trading_data(ticker_list, start_date, end_date, index_as_date = True, interval = "1d")

#Select stocks based on attributes/performance for investment or trading
tradin_select, _, volatl_select = get_selection(trading_datas)
if len(tradin_select) == 0:
  tradin_select = volatl_select


output_text = str(grouped_stocks) +  "_watchlist.txt"
with open('/content/drive/MyDrive/Colab Notebooks/{}'.format(str(output_text)), "w") as text_file:
  print(tradin_select, file=text_file)


ticker_list = volatl_select 
start_date = (datetime.date.today() + datetime.timedelta(days=-7300)).strftime("%m/%d/%Y")                                                            #Input format is: month/day/Year
end_date = datetime.date.today().strftime("%m/%d/%Y")
trading_datas = get_trading_data(ticker_list, start_date, end_date, index_as_date = True, interval = "1d")



#############################################TRADING#####################################################################################################################
#Call algotrading methods for selected stocks for Day-Trading
fastm, slowm, smooth = 6, 17, 10                    #MACD parameters
roll_period = 4                                     #Bollinger Bands parameter
roll_low, roll_high, fasts, slows = 5, 14, 7, 14    #Stochastic bands parameters
roll_wks1, roll_wks2, roll_wks10, roll_wks30, roll_wks40 = 5, 10, 50, 150, 200      #Simple moving average parameters

macd_datat, boband_datat, stochst_datat, smap_datat, smav_datat = {}, {}, {}, {}, {}

for select_ticker in tradin_select:
  macd_datat[select_ticker] = get_macd(trading_datas, select_ticker, fastm, slowm, smooth)
  boband_datat[select_ticker] = get_boband(trading_datas, select_ticker, roll_period)
  stochst_datat[select_ticker] = get_stochst(trading_datas, select_ticker, roll_low, roll_high, fasts, slows)
  smap_datat[select_ticker] = get_smap(trading_datas, select_ticker, roll_wks1, roll_wks2, roll_wks10, roll_wks30, roll_wks40)
  smav_datat[select_ticker] = get_smav(trading_datas, select_ticker, roll_wks1, roll_wks2, roll_wks10, roll_wks30, roll_wks40)  

macd_datat = pd.concat(macd_datat).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')
boband_datat = pd.concat(boband_datat).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')
stochst_datat = pd.concat(stochst_datat).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')
smap_datat = pd.concat(smap_datat).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')
smav_datat = pd.concat(smav_datat).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')

#Evaluate the strategy of each trading decision method
macd_decisiont, smap_decisiont, smav_decisiont, stochst_decisiont, boband_decisiont = {}, {}, {}, {}, {}
for select_ticker in tradin_select:
  macd_decisiont[select_ticker] = strategy_macd(macd_data = macd_datat[macd_datat['ticker'] == select_ticker])
  smap_decisiont[select_ticker] = strategy_smap(smap_data = smap_datat[smap_datat['ticker'] == select_ticker])
  smav_decisiont[select_ticker] = strategy_smav(smav_data = smav_datat[smav_datat['ticker'] == select_ticker])
  stochst_decisiont[select_ticker] = strategy_stochst(stochst_data = stochst_datat[stochst_datat['ticker'] == select_ticker])
  boband_decisiont[select_ticker] = strategy_boband(boband_data = boband_datat[boband_datat['ticker'] == select_ticker])

macd_decisiont = pd.concat(macd_decisiont).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')
boband_decisiont = pd.concat(boband_decisiont).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')
stochst_decisiont = pd.concat(stochst_decisiont).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')
smap_decisiont = pd.concat(smap_decisiont).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')
smav_decisiont = pd.concat(smav_decisiont).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')

#Evaluate trading scenarios
tradin_table = {}
for select_ticker in tradin_select:
  tradin_table[select_ticker] = scenarios_tradin(macd_decisiont[macd_decisiont['ticker'] == select_ticker], smap_decisiont[smap_decisiont['ticker'] == select_ticker],\
                                                 smav_decisiont[smav_decisiont['ticker'] == select_ticker], stochst_decisiont[stochst_decisiont['ticker'] == select_ticker],\
                                                 boband_decisiont[boband_decisiont['ticker'] == select_ticker])

tradin_table = pd.concat(tradin_table).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')

########################################################################################################################################################################



########################################################BACK TESTING####################################################################################################
#Trading
SV_return = sum(tradin_table['close']*tradin_table['SV_signal'])*(-1)            #multiply by -1 so that profit is +ve and loss is -ve
MV_return = sum(tradin_table['close']*tradin_table['MV_signal'])*(-1)
LV_return = sum(tradin_table['close']*tradin_table['LV_signal'])*(-1)

backtest_tradin = {'short view': SV_return, 'medium view': MV_return, 'long view': LV_return}

output_text = str(grouped_stocks) +  "_backtest.txt"
with open('/content/drive/MyDrive/Colab Notebooks/{}'.format(str(output_text)), "w") as text_file:
  print(backtest_tradin, file=text_file)

np.count_nonzero(tradin_table['SV_signal']>0)

In [ ]:
"""
WORKFLOW EXECUTION SCRIPT: THIS IS USED TO RUN THE TRADING MODULES ON REAL-TIME BASIS

"""
#Selection of stocks for trading or investment
ticker_list = get_ticker_list(grouped_stocks='tsx')                                             #Other currently available groups are: 'tsx', 'sp500', 'nasdaq' 

#Get (historical) data on longterm performance os stock - 20 years preferable
start_date = (datetime.date.today() + datetime.timedelta(days=-366)).strftime("%m/%d/%Y")                                                            #Input format is: month/day/Year
end_date = (datetime.date.today() + datetime.timedelta(days=-1)).strftime("%m/%d/%Y")
trading_datas = get_trading_data(ticker_list, start_date, end_date, index_as_date = True, interval = "1d")

#Select stocks based on attributes/performance for investment or trading
tradin_select, _, volatl_select = get_selection(trading_datas)
if len(tradin_select) == 0:
  tradin_select = volatl_select


#Get (real-time) data for trading decisions
tradin_data = {}

for ticker in tradin_select:                                                                     #volatl_select could be used if tradin_select is empty
  tradin_data[ticker] = yf.download(ticker, period = "1d", interval = "5m", auto_adjust=True)

tradin_data = pd.concat(tradin_data)

tradin_data = tradin_data.reset_index()\
                         .rename(columns={'level_0':'ticker'})\
                         .rename(columns={'Datetime':'date'})\
                         .set_index('date')\
                         .rename(columns={'Close':'close'})\
                         .rename(columns={'Open':'open'})\
                         .rename(columns={'High':'high'})\
                         .rename(columns={'Low':'low'})\
                         .rename(columns={'Volume':'volume'})\
                         .dropna()

#Combine the realtime (tradin_data) and historical (trading_datas) trading data get day-trading data
daytrading_data = {}
for ticker in tradin_select:
  daytrading_data[ticker] = trading_datas[trading_datas['ticker'] == ticker].drop(labels=['adjclose', 'prange'], axis =1)

daytrading_data = pd.concat(daytrading_data)
daytrading_data = daytrading_data.reset_index()\
                                 .drop(labels = 'level_0', axis = 1)\
                                 .set_index('date')

daytrading_data = daytrading_data.append(tradin_data)


#############################################TRADING########################################################################
#Call algotrading methods for selected stocks for Day-Trading
fastm, slowm, smooth = 6, 17, 10                    #MACD parameters
roll_period = 4                                     #Bollinger Bands parameter
roll_low, roll_high, fasts, slows = 5, 14, 7, 14    #Stochastic bands parameters
roll_wks1, roll_wks2, roll_wks10, roll_wks30, roll_wks40 = 5, 10, 50, 150, 200      #Simple moving average parameters
trading_datas = daytrading_data

macd_datat, boband_datat, stochst_datat, smap_datat, smav_datat = {}, {}, {}, {}, {}

for select_ticker in tradin_select:
  macd_datat[select_ticker] = get_macd(trading_datas, select_ticker, fastm, slowm, smooth)
  boband_datat[select_ticker] = get_boband(trading_datas, select_ticker, roll_period)
  stochst_datat[select_ticker] = get_stochst(trading_datas, select_ticker, roll_low, roll_high, fasts, slows)
  smap_datat[select_ticker] = get_smap(trading_datas, select_ticker, roll_wks1, roll_wks2, roll_wks10, roll_wks30, roll_wks40)
  smav_datat[select_ticker] = get_smav(trading_datas, select_ticker, roll_wks1, roll_wks2, roll_wks10, roll_wks30, roll_wks40)  

macd_datat = pd.concat(macd_datat).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')
boband_datat = pd.concat(boband_datat).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')
stochst_datat = pd.concat(stochst_datat).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')
smap_datat = pd.concat(smap_datat).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')
smav_datat = pd.concat(smav_datat).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')

#Evaluate the strategy of each trading decision method
macd_decisiont, smap_decisiont, smav_decisiont, stochst_decisiont, boband_decisiont = {}, {}, {}, {}, {}
for select_ticker in tradin_select:
  macd_decisiont[select_ticker] = strategy_macd(macd_data = macd_datat[macd_datat['ticker'] == select_ticker])
  smap_decisiont[select_ticker] = strategy_smap(smap_data = smap_datat[smap_datat['ticker'] == select_ticker])
  smav_decisiont[select_ticker] = strategy_smav(smav_data = smav_datat[smav_datat['ticker'] == select_ticker])
  stochst_decisiont[select_ticker] = strategy_stochst(stochst_data = stochst_datat[stochst_datat['ticker'] == select_ticker])
  boband_decisiont[select_ticker] = strategy_boband(boband_data = boband_datat[boband_datat['ticker'] == select_ticker])

macd_decisiont = pd.concat(macd_decisiont).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')
boband_decisiont = pd.concat(boband_decisiont).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')
stochst_decisiont = pd.concat(stochst_decisiont).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')
smap_decisiont = pd.concat(smap_decisiont).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')
smav_decisiont = pd.concat(smav_decisiont).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')

#Evaluate trading scenarios
tradin_table = {}
for select_ticker in tradin_select:
  tradin_table[select_ticker] = scenarios_tradin(macd_decisiont[macd_decisiont['ticker'] == select_ticker], smap_decisiont[smap_decisiont['ticker'] == select_ticker],\
                                                 smav_decisiont[smav_decisiont['ticker'] == select_ticker], stochst_decisiont[stochst_decisiont['ticker'] == select_ticker],\
                                                 boband_decisiont[boband_decisiont['ticker'] == select_ticker])

tradin_table = pd.concat(tradin_table).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')

###############################################################################################################################################



########################################################BACK TESTING####################################################################################################
#Trading
SV_return = sum(tradin_table['close']*tradin_table['SV_signal'])*(-1)            #multiply by -1 so that profit is +ve and loss is -ve
MV_return = sum(tradin_table['close']*tradin_table['MV_signal'])*(-1)
LV_return = sum(tradin_table['close']*tradin_table['LV_signal'])*(-1)

backtest_tradin = [SV_return, MV_return, LV_return]

np.count_nonzero(tradin_table['SV_signal']>0)

/usr/local/lib/python3.7/dist-packages/yahoo_fin/stock_info.py:116: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[*********************100%***********************]  1 of 1 completed

1 Failed download:
- DXG: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


KeyError: ignored

In [ ]:
"""
WORKFLOW EXECUTION SCRIPT: THIS IS USED TO RUN THE INVESTING MODULES ON REAL-TIME BASIS

"""
#Selection of stocks for trading or investment
ticker_list = get_ticker_list(grouped_stocks='tsx60')                                             #Other currently available groups are: 'tsx', 'sp500', 'nasdaq' 

#Get (historical) data on longterm performance os stock - 20 years preferable
start_date = (datetime.date.today() + datetime.timedelta(days=-366)).strftime("%m/%d/%Y")                                                            #Input format is: month/day/Year
end_date = (datetime.date.today() + datetime.timedelta(days=-1)).strftime("%m/%d/%Y")
trading_datas = get_trading_data(ticker_list, start_date, end_date, index_as_date = True, interval = "1d")

#Select stocks based on attributes/performance for investment or trading
_, invest_select, _ = get_selection(trading_datas)


#Get (real-time) data for investment decisions
invest_data = {}

for ticker in invest_select:
  invest_data[ticker] = yf.download(ticker, period = "1d", interval = "5m", auto_adjust=True)

invest_data = pd.concat(invest_data)

invest_data = invest_data.reset_index()\
                         .rename(columns={'level_0':'ticker'})\
                         .rename(columns={'Datetime':'date'})\
                         .set_index('date')\
                         .rename(columns={'Close':'close'})\
                         .rename(columns={'Open':'open'})\
                         .rename(columns={'High':'high'})\
                         .rename(columns={'Low':'low'})\
                         .rename(columns={'Volume':'volume'})\
                         .dropna()

#Combine the realtime (invest_data) and historical (trading_datas) trading data get day-investing data
dayinvesting_data = {}
for ticker in invest_select:
  dayinvesting_data[ticker] = trading_datas[trading_datas['ticker'] == ticker].drop(labels=['adjclose', 'prange'], axis =1)

dayinvesting_data = pd.concat(dayinvesting_data)
dayinvesting_data = dayinvesting_data.reset_index()\
                                     .drop(labels = 'level_0', axis = 1)\
                                     .set_index('date')

dayinvesting_data = dayinvesting_data.append(invest_data)


###############################################INVESTING#######################################################################################
#Call algotrading methods for selected stocks for Day-Investing
fastm, slowm, smooth = 6, 17, 10                                       #MACD parameters
roll_period = 4                                                        #Bollinger Bands parameter
roll_low, roll_high, fasts, slows = 5, 14, 7, 14                       #Stochastic bands parameters
roll_wks1, roll_wks2, roll_wks10, roll_wks30, roll_wks40 = 5, 10, 50, 150, 200       #Simple moving average parameters
trading_datas = dayinvesting_data

macd_datai, boband_datai, stochst_datai, smap_datai, smav_datai = {}, {}, {}, {}, {}

for select_ticker in invest_select:
  macd_datai[select_ticker] = get_macd(trading_datas, select_ticker, fastm, slowm, smooth)
  boband_datai[select_ticker] = get_boband(trading_datas, select_ticker, roll_period)
  stochst_datai[select_ticker] = get_stochst(trading_datas, select_ticker, roll_low, roll_high, fasts, slows)
  smap_datai[select_ticker] = get_smap(trading_datas, select_ticker, roll_wks1, roll_wks2, roll_wks10, roll_wks30, roll_wks40)
  smav_datai[select_ticker] = get_smav(trading_datas, select_ticker, roll_wks1, roll_wks2, roll_wks10, roll_wks30, roll_wks40)  

macd_datai = pd.concat(macd_datai).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')
boband_datai = pd.concat(boband_datai).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')
stochst_datai = pd.concat(stochst_datai).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')
smap_datai = pd.concat(smap_datai).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')
smav_datai = pd.concat(smav_datai).reset_index().drop(labels = 'level_0', axis = 1).set_index('date')


#Evaluate the strategy of each investing decision method
macd_decisioni, smap_decisioni, smav_decisioni, stochst_decisioni, boband_decisioni = {}, {}, {}, {}, {}
for select_ticker in invest_select:
  macd_decisioni[select_ticker] = strategy_macd(macd_data = macd_datai[macd_datai['ticker'] == select_ticker])
  smap_decisioni[select_ticker] = strategy_smap(smap_data = smap_datai[smap_datai['ticker'] == select_ticker])
  smav_decisioni[select_ticker] = strategy_smav(smav_data = smav_datai[smav_datai['ticker'] == select_ticker])
  stochst_decisioni[select_ticker] = strategy_stochst(stochst_data = stochst_datai[stochst_datai['ticker'] == select_ticker])
  boband_decisioni[select_ticker] = strategy_boband(boband_data = boband_datai[boband_datai['ticker'] == select_ticker])

macd_decisioni = pd.concat(macd_decisioni).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')
boband_decisioni = pd.concat(boband_decisioni).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')
stochst_decisioni = pd.concat(stochst_decisioni).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')
smap_decisioni = pd.concat(smap_decisioni).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')
smav_decisioni = pd.concat(smav_decisioni).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')

#Evaluate investing scenarios
invest_table = {}
for select_ticker in invest_select:
  invest_table[select_ticker] = scenarios_invest(macd_decisioni[macd_decisioni['ticker'] == select_ticker], smap_decisioni[smap_decisioni['ticker'] == select_ticker],\
                                                 smav_decisioni[smav_decisioni['ticker'] == select_ticker], stochst_decisioni[stochst_decisioni['ticker'] == select_ticker],\
                                                 boband_decisioni[boband_decisioni['ticker'] == select_ticker])

invest_table = pd.concat(invest_table).reset_index().rename(columns = {'level_0':'ticker'}).set_index('date')

########################################################################################################################################################################



########################################################BACK TESTING####################################################################################################
#Investing
HR_return = sum(invest_table['close']*invest_table['HR_signal'])*(-1)            #multiply by -1 so that profit is +ve and loss is -ve
MR_return = sum(invest_table['close']*invest_table['MR_signal'])*(-1)
LR_return = sum(invest_table['close']*invest_table['LR_signal'])*(-1)


backtest_invest = [HR_return, MR_return, LR_return]

np.count_nonzero(invest_table['LR_signal']>0)

In [ ]:
def plot_macd(macd_data):

  macd_decisions = strategy_macd(macd_data)
  buy_price = macd_decisions['buy_price']
  sell_price = macd_decisions['sell_price']

  ax1 = plt.subplot2grid((8,1), (0,0), rowspan = 5, colspan = 1)
  ax2 = plt.subplot2grid((8,1), (5,0), rowspan = 3, colspan = 1)

  ax1.plot(macd_data['price'], color = 'skyblue', linewidth = 2, label = str(macd_data['ticker'][1]))
  ax1.plot(macd_data.index, buy_price, marker = '^', color = 'green', markersize = 10, label = 'BUY SIGNAL', linewidth = 0)
  ax1.plot(macd_data.index, sell_price, marker = 'v', color = 'r', markersize = 10, label = 'SELL SIGNAL', linewidth = 0)
  ax1.legend()
  ax1.set_title(str(macd_data['ticker'][1]) + ' MACD SIGNALS')
  ax2.plot(macd_data['macd'], color = 'grey', linewidth = 1.5, label = 'MACD')
  ax2.plot(macd_data['signal'], color = 'skyblue', linewidth = 1.5, label = 'SIGNAL')

  for i in range(len(macd_data)):
    if macd_data['hist'][i] < 0:
      ax2.bar(macd_data.index[i], macd_data['hist'][i], color = '#ef5350')
    else:
      ax2.bar(macd_data.index[i], macd_data['hist'][i], color = '#26a69a')

  plt.legend(loc = 'lower right') 
 


In [ ]:
def plot_bband(trade_data):
#This figure is designed for live trading. Live share price evolution data is supplied at 1 minute intervals.

#declare figure
  fig = go.Figure()

  fig.add_trace(go.Scatter(x=trade_data.index, y= trade_data['Mband'],line=dict(color='blue', width=.7), name = 'Middle Band'))
  fig.add_trace(go.Scatter(x=trade_data.index, y= trade_data['Uband'],line=dict(color='red', width=1.5), name = 'Upper Band (Sell)'))
  fig.add_trace(go.Scatter(x=trade_data.index, y= trade_data['Lband'],line=dict(color='green', width=1.5), name = 'Lower Band (Buy)'))


#Candlestick
  fig.add_trace(go.Candlestick(x=trade_data.index,
                               open=trade_data['open'],
                               high=trade_data['high'],
                               low=trade_data['low'],
                               close=trade_data['close'], name = 'market data'))

# Add titles
  fig.update_layout(
      title= trade_data['ticker'][1]) + ' live share price evolution',
      yaxis_title='Stock Price ($ per share)')

# X-Axes
  fig.update_xaxes(
      rangeslider_visible=True,ç
      rangeselector=dict(
          buttons=list([
                        dict(count=15, label="15m", step="minute", stepmode="backward"),
                        dict(count=45, label="45m", step="minute", stepmode="backward"),
                        dict(count=1, label="HTD", step="hour", stepmode="todate"),
                        dict(count=3, label="3h", step="hour", stepmode="backward"),
                        dict(step="all")
                        ])
          )
      )
  
#Show 
  fig.show()
  

# **Unit Testing of Code**

Testing components of code
Code dumps are located below

In [ ]:
ticker_list = get_ticker_list(grouped_stocks='dow')


#trading_datas, *_  = get_historical_data(ticker_list, start_date = "01/01/2010", end_date = "01/01/2021", index_as_date = True, interval = "1mo")
trading_datas, _ , _ , _ = get_historical_data(ticker_list="AAPL", start_date = "05/10/2021", end_date = "05/11/2021", index_as_date = True, interval = "1d")
trading_datas = yf.download(ticker_list, period = "1d", interval = "60m", auto_adjust=True)  #check that downloaded table is same format with the table from YahooFinancial library (as against yfinance)
trading_datas = get_realtime_data2(ticker_list = ['AAPL', 'CVX'], start="2021-05-11", end="2021-05-12", interval = "60m")
trading_datas = yf.download(tickers=ticker_list, start="2021-05-11", end="2021-05-12", interval = "60m", auto_adjust=True)

del trading_datas

trading_datas.T

tradin_select, invest_select, volatl_select = get_selection(trading_datas)
tradin_select
invest_select
volatl_select

macd_data = get_macd(trading_datas, select_ticker = "AAPL", fast=6, slow=17, smooth=10)

plot_macd(macd_data)

current_position = positions_table(macd_data)
current_position


buy_price, sell_price, macd_signal = strategy_macd(macd_data)


In [ ]:
def get_historical_data(ticker_list, start_date, end_date, index_as_date, interval):
  
  trading_datas = {}
  
  for ticker in ticker_list:
    trading_datas[ticker] = si.get_data(ticker, start_date, end_date, index_as_date, interval)  

  trading_datas = pd.concat(trading_datas)

  return trading_datas  



In [ ]:
def get_realtime_data(ticker_list, period, interval):

  trading_datas = {}

  for ticker in ticker_list:
    trading_datas[ticker] = yf.download(ticker, period, interval)  #check that downloaded table is same format with the table from YahooFinancial library (as against yfinance)

  trading_datas = pd.concat(trading_datas)

  return trading_datas
  

In [ ]:
def positions_table(macd_data):

  *_ , macd_signal  = strategy_macd(macd_data)

  position = []
  for i in range(len(macd_signal)):
    if macd_signal[i] > 1:
      position.append(0)
    else:
      position.append(1)
        
  for i in range(len(macd_data['price'])):
    if macd_signal[i] == 1:
        position[i] = 1
    elif macd_signal[i] == -1:
        position[i] = 0
    else:
        position[i] = position[i-1]

  macd_signal = pd.DataFrame(macd_signal).rename(columns = {0:'macd_signal'}).set_index(macd_data.index)
  position = pd.DataFrame(position).rename(columns = {0:'macd_position'}).set_index(macd_data.index)

  position_table = pd.concat([macd_data['price'], macd_data['macd'], macd_data['signal'], macd_signal, position], join = 'inner', axis = 1)
  
  return position_table

In [ ]:

start_date = "01/01/2021"
#end_date=datetime.date.today().isoformat()
end_date=datetime.date.today().strftime("%d/%m/%y")  #today
end_date=(datetime.date.today() + datetime.timedelta(days=1)).strftime("%d/%m/%y")  #tomorrow

end_date=datetime.date.today().strftime("%m-%d-%Y")  #today
end_date=(datetime.date.today() + datetime.timedelta(days=1)).strftime("%m-%d-%Y")  #tomorrow

end_date=datetime.date.today().strftime("%Y-%m-%d")  #today
end_date=(datetime.date.today() + datetime.timedelta(days=-1)).strftime("%Y-%m-%d")  #yesterday

df['date'] = pd.to_datetime(df.index).time
df.set_index('date', inplace=True)

drop(labels=['Adj Close', 'Dividends', 'Stock Splits'], axis=1)

df3 = get_smap(trading_datas, select_ticker = "CVX", rolling_period=10)

plot_macd(df['close'], macd_data['macd'], macd_data['signal'], macd_data['hist'])

'2021-05-17'

In [ ]:
dow_list = si.tickers_dow()
snp_list = si.tickers_sp500()
nas_list = si.tickers_nasdaq()

tsx_list = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tsx_tickers.csv')
tsx_list = tsx_list.iloc[:,0].tolist()


function ClickConnect(){
    console.log("Clicked on connect button"); 
    document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect,60000)


In [ ]:
ticker_list = dow_list
trading_datas = {}
cashflow_datas = {}
balncsheet_datas = {}
incomstment_datas = {}
for ticker in ticker_list:
    trading_datas[ticker] = si.get_data(ticker, start_date = "01/01/2001", end_date = "01/01/2021", index_as_date = True, interval = "1d")  
    cashflow_datas[ticker] = si.get_cash_flow(ticker)  
    balncsheet_datas[ticker] = si.get_balance_sheet(ticker)  
    incomstment_datas[ticker] = si.get_income_statement(ticker)  

trading_datas = pd.concat(trading_datas)
cashflow_datas = pd.concat(cashflow_datas)  
balncsheet_datas = pd.concat(balncsheet_datas)
incomstment_datas = pd.concat(incomstment_datas)

cashflow_datas = cashflow_datas.transpose()
balncsheet_datas = balncsheet_datas.transpose()
incomstment_datas = incomstment_datas.transpose()

 

intervals -> 1d, 1wk, 1mo, 3mo, 1yr/1y(?)